In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

C:\Users\Baha Tegar\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
from google.cloud import bigquery
from google.oauth2 import service_account

In [3]:
key_path = './service_account/gentle-keyword-423715-j0-03be08ad6412.json'

credentials = service_account.Credentials.from_service_account_file(
    key_path,
    scopes=["https://www.googleapis.com/auth/bigquery"]
)

In [4]:
from google.cloud import bigquery

client = bigquery.Client(
    credentials = credentials,
    project=credentials.project_id
)

In [5]:
query = """
SELECT 
  oi.order_id,
  oi.id order_items_id,
  oi.user_id,
  oi.product_id,
  p.name,
  p.brand, 
  p.category, 
  p.cost, 
  p.retail_price,
  oi.sale_price,
  oi.sale_price * o.num_of_item totalSales,
  (oi.sale_price - p.cost) * o.num_of_item totalRevenue,
  o.gender,
  o.created_at,
  o.num_of_item
FROM `bigquery-public-data.thelook_ecommerce.orders` o
  INNER JOIN `bigquery-public-data.thelook_ecommerce.order_items` oi
    ON o.order_id = oi.order_id
  INNER JOIN `bigquery-public-data.thelook_ecommerce.products` p
    ON oi.product_id = p.id
WHERE 
  o.status = 'Complete'
  AND
  (EXTRACT(YEAR FROM o.created_at) >= 2023)
"""

df = client.query(query).to_dataframe()
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27873 entries, 0 to 27872
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   order_id        27873 non-null  Int64              
 1   order_items_id  27873 non-null  Int64              
 2   user_id         27873 non-null  Int64              
 3   product_id      27873 non-null  Int64              
 4   name            27871 non-null  object             
 5   brand           27846 non-null  object             
 6   category        27873 non-null  object             
 7   cost            27873 non-null  float64            
 8   retail_price    27873 non-null  float64            
 9   sale_price      27873 non-null  float64            
 10  totalSales      27873 non-null  float64            
 11  totalRevenue    27873 non-null  float64            
 12  gender          27873 non-null  object             
 13  created_at      27873 non-null 

,order_id,order_items_id,user_id,product_id,name,brand,category,cost,retail_price,sale_price,totalSales,totalRevenue,gender,created_at,num_of_item
0,37907,54971,30352,8028,Collections Etc - Autumn Floral V Neck Top And...,Samanthas Style Shoppe,Clothing Sets,20.804071,32.970001,32.970001,32.970001,12.165931,F,2023-08-30 03:33:00+00:00,1
1,82460,119565,66154,8032,Aeropostale Womens; Juniors Long Sleeve Logo G...,Aeropostale,Clothing Sets,21.010500,34.500000,34.500000,103.500000,40.468500,F,2023-09-07 05:49:00+00:00,3
2,27440,39806,22021,8037,Jostar Short Sleeve Solid Stretchy Capri Pants...,Jostar,Clothing Sets,25.538451,38.990002,38.990002,38.990002,13.451551,F,2024-03-06 03:48:00+00:00,1
3,66661,96659,53461,8037,Jostar Short Sleeve Solid Stretchy Capri Pants...,Jostar,Clothing Sets,25.538451,38.990002,38.990002,77.980003,26.903101,F,2024-03-30 08:07:00+00:00,2
4,95469,138528,76631,8037,Jostar Short Sleeve Solid Stretchy Capri Pants...,Jostar,Clothing Sets,25.538451,38.990002,38.990002,77.980003,26.903101,F,2023-12-23 05:02:00+00:00,2


In [6]:
df['order_id'] = df['order_id'].astype(str)
df['order_items_id'] = df['order_items_id'].astype(str)
df['user_id'] = df['user_id'].astype(str)
df['product_id'] = df['product_id'].astype(str)

print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27873 entries, 0 to 27872
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   order_id        27873 non-null  object             
 1   order_items_id  27873 non-null  object             
 2   user_id         27873 non-null  object             
 3   product_id      27873 non-null  object             
 4   name            27871 non-null  object             
 5   brand           27846 non-null  object             
 6   category        27873 non-null  object             
 7   cost            27873 non-null  float64            
 8   retail_price    27873 non-null  float64            
 9   sale_price      27873 non-null  float64            
 10  totalSales      27873 non-null  float64            
 11  totalRevenue    27873 non-null  float64            
 12  gender          27873 non-null  object             
 13  created_at      27873 non-null 

In [7]:
data = df.copy()
print(data['user_id'].nunique())
# data = df
print(data.info())
print("Number of unique items: ", data['name'].nunique())
data.describe()

17464
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27873 entries, 0 to 27872
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   order_id        27873 non-null  object             
 1   order_items_id  27873 non-null  object             
 2   user_id         27873 non-null  object             
 3   product_id      27873 non-null  object             
 4   name            27871 non-null  object             
 5   brand           27846 non-null  object             
 6   category        27873 non-null  object             
 7   cost            27873 non-null  float64            
 8   retail_price    27873 non-null  float64            
 9   sale_price      27873 non-null  float64            
 10  totalSales      27873 non-null  float64            
 11  totalRevenue    27873 non-null  float64            
 12  gender          27873 non-null  object             
 13  created_at      27873 non

,cost,retail_price,sale_price,totalSales,totalRevenue,num_of_item
count,27873.000000,27873.000000,27873.000000,27873.000000,27873.000000,27873.0
mean,28.890783,60.071805,60.071805,113.681582,59.013024,1.89097
std,31.152191,66.890414,66.890414,159.917014,86.622715,1.052437
min,0.008300,0.020000,0.020000,0.020000,0.011700,1.0
25%,11.350000,24.500000,24.500000,33.990002,16.835791,1.0
50%,20.000000,39.990002,39.990002,66.000000,33.157000,2.0
75%,35.035619,69.989998,69.989998,135.000000,68.408000,2.0
max,557.151002,990.000000,990.000000,3612.000000,2141.915995,4.0


In [8]:
# Time splitting 
n_days = 90
max_date = data['created_at'].max()
cutoff = max_date - pd.to_timedelta(n_days, unit='d')

temp_in = data[data['created_at'] <= cutoff]
temp_out = data[data['created_at'] > cutoff]

In [9]:
# Feature engineering (RFM)

targets = temp_out\
            .groupby('user_id')\
            .agg({'totalRevenue': 'sum'})\
            .rename({'totalRevenue': 'spend_90_total'}, axis=1)

targets

,spend_90_total
user_id,
10004,31.068901
10013,89.648000
10040,53.336510
10095,103.412562
10105,54.531000
...,...
99916,139.010316
99949,37.666551
99955,52.270000


In [10]:
# Feature Engineering

# Recency

recency_feature = temp_in[['user_id', 'created_at']]\
                    .groupby('user_id')\
                    .apply(lambda x: (max_date - x['created_at'].max()) / pd.to_timedelta(1, 'day'), include_groups=False)\
                    .to_frame()\
                    .set_axis(['recency'], axis=1)

recency_feature

,recency
user_id,
1000,96.373841
10004,357.347452
10007,469.141896
10009,320.507174
10020,240.466202
...,...
99971,151.787730
99986,118.304396
99987,376.609952


In [11]:
# Frequency

frequency_feature = temp_in[['user_id', 'created_at']]\
                    .groupby('user_id')\
                    .count()\
                    .set_axis(['frequency'], axis=1)

frequency_feature

,frequency
user_id,
1000,1
10004,1
10007,4
10009,2
10020,2
...,...
99971,2
99986,1
99987,1


In [12]:
# monetary

monetary_feature = temp_in\
                    .groupby('user_id')\
                    .agg({ 'totalRevenue': ['sum', 'mean'] })\
                    .set_axis(['monetary_sum', 'monetary_mean'], axis=1)

monetary_feature

,monetary_sum,monetary_mean
user_id,,
1000,109.200000,109.200000
10004,22.594000,22.594000
10007,549.195908,137.298977
10009,61.007580,30.503790
10020,99.212000,49.606000
...,...,...
99971,187.302402,93.651201
99986,5.636150,5.636150
99987,39.780000,39.780000


In [13]:
# Combine feature

features = pd.concat([recency_feature, frequency_feature, monetary_feature], axis=1)\
            .merge(targets,
                  left_index=True,
                  right_index=True,
                  how='left')\
            .fillna(0)

features

,recency,frequency,monetary_sum,monetary_mean,spend_90_total
user_id,,,,,
1000,96.373841,1,109.200000,109.200000,0.000000
10004,357.347452,1,22.594000,22.594000,31.068901
10007,469.141896,4,549.195908,137.298977,0.000000
10009,320.507174,2,61.007580,30.503790,0.000000
10020,240.466202,2,99.212000,49.606000,0.000000
...,...,...,...,...,...
99971,151.787730,2,187.302402,93.651201,0.000000
99986,118.304396,1,5.636150,5.636150,0.000000
99987,376.609952,1,39.780000,39.780000,0.000000


In [14]:
# Machine learning (regression)

from xgboost import XGBClassifier, XGBRegressor

from sklearn.model_selection import GridSearchCV

X = features[['recency', 'frequency', 'monetary_sum', 'monetary_mean']]
y_spending = features['spend_90_total']

xgb_reg = XGBRegressor(objective='reg:squarederror', random_state=123)

xgb_reg_model = GridSearchCV(estimator=xgb_reg, 
                             param_grid=dict(learning_rate=[0.01, 0.1, 0.3, 0.5]),
                             scoring='neg_mean_absolute_error',
                             refit=True,
                             cv=5)

xgb_reg_model.fit(X, y_spending)

GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None, gpu_id=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, max_bin=None,
                                    max_cat_threshold=None,
                                    max_cat_to_onehot=None, max_delta_step=None,
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None, n_estimators=100,
                                    n_jobs=None, num_parallel_tree=None,
                                    predictor=None, random_state=123, ...),
             param_grid={'learning_rate': [0.01, 0.1, 0.3, 0.5]},
             scoring='neg_mean_absolute_error')

In [15]:
print("Best score: ", xgb_reg_model.best_score_)
print("Best params: ", xgb_reg_model.best_params_)
print("Best estimator: \n", xgb_reg_model.best_estimator_)

predictions_reg = xgb_reg_model.predict(X)

Best score:  -6.265758518651426
Best params:  {'learning_rate': 0.01}
Best estimator: 
 XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.01, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=123, ...)


In [16]:
# Feature important

imp_spend_amount_dict = xgb_reg_model \
    .best_estimator_ \
    .get_booster() \
    .get_score(importance_type = 'gain') 

imp_spend_amount_dict

{'recency': 56895.0078125,
 'frequency': 5539.3974609375,
 'monetary_sum': 81080.5,
 'monetary_mean': 29337.7578125}

In [17]:
predictions_df = pd.concat(
    [
        features.reset_index(),
        pd.DataFrame(predictions_reg).set_axis(['pred_spend'], axis=1),
    ], 
    axis=1
)

predictions_df

,user_id,recency,frequency,monetary_sum,monetary_mean,spend_90_total,pred_spend
0,1000,96.373841,1,109.200000,109.200000,0.000000,4.178180
1,10004,357.347452,1,22.594000,22.594000,31.068901,1.722428
2,10007,469.141896,4,549.195908,137.298977,0.000000,1.773968
3,10009,320.507174,2,61.007580,30.503790,0.000000,1.741930
4,10020,240.466202,2,99.212000,49.606000,0.000000,2.617600
...,...,...,...,...,...,...,...
12068,99971,151.787730,2,187.302402,93.651201,0.000000,3.566005
12069,99986,118.304396,1,5.636150,5.636150,0.000000,2.210721
12070,99987,376.609952,1,39.780000,39.780000,0.000000,1.781610
12071,99988,212.249535,1,97.812000,97.812000,0.000000,3.189983


In [18]:
# import pickle
# # Save the model to a file
# with open('./model/model_xgb_reg.pkl', 'wb') as model_file:
#     pickle.dump(xgb_reg_model, model_file)

In [19]:
# Predict next 90 days

recency_feature = data[['user_id', 'created_at']]\
                    .groupby('user_id')\
                    .apply(lambda x: (max_date - x['created_at'].max()) / pd.to_timedelta(1, 'day'), include_groups=False)\
                    .to_frame()\
                    .set_axis(['recency'], axis=1)

frequency_feature = data[['user_id', 'created_at']]\
                    .groupby('user_id')\
                    .count()\
                    .set_axis(['frequency'], axis=1)

monetary_feature = data[['user_id', 'totalRevenue']]\
                    .groupby('user_id')\
                    .agg({ 'totalRevenue': ['sum', 'mean'] })\
                    .set_axis(['monetary_sum', 'monetary_mean'], axis=1)


features = pd.concat([recency_feature, frequency_feature, monetary_feature], axis=1)\
            .fillna(0)

features

,recency,frequency,monetary_sum,monetary_mean
user_id,,,,
1000,96.373841,1,109.200000,109.200000
10004,60.347452,2,53.662901,26.831450
10007,469.141896,4,549.195908,137.298977
10009,320.507174,2,61.007580,30.503790
10013,34.686341,1,89.648000,89.648000
...,...,...,...,...
99975,26.139813,1,6.775450,6.775450
99986,118.304396,1,5.636150,5.636150
99987,376.609952,1,39.780000,39.780000


In [20]:
prediction_revenue = xgb_reg_model.predict(features)

In [21]:
result = pd.DataFrame(index=features.index)
result['pred_next90_revenue'] = prediction_revenue

result = result.reset_index()
result

,user_id,pred_next90_revenue
0,1000,4.178180
1,10004,0.448219
2,10007,1.773968
3,10009,1.741930
4,10013,0.448219
...,...,...
17459,99975,1.679508
17460,99986,2.210721
17461,99987,1.781610
17462,99988,3.189983


In [22]:
# result.to_csv('pred-revenue-next90days.csv', index=False)

In [25]:
data.set_index("created_at").resample('ME').agg({"totalRevenue": 'sum'})

,totalRevenue
created_at,
2023-01-31 00:00:00+00:00,51636.038174
2023-02-28 00:00:00+00:00,50384.202680
2023-03-31 00:00:00+00:00,52697.171393
2023-04-30 00:00:00+00:00,57201.724886
2023-05-31 00:00:00+00:00,62591.078277
2023-06-30 00:00:00+00:00,57968.263822
2023-07-31 00:00:00+00:00,78143.717445
2023-08-31 00:00:00+00:00,80262.676879
2023-09-30 00:00:00+00:00,79320.929587


In [27]:
data['revenue2'] = (data['sale_price'] - data['cost']) * data['num_of_item']

data.set_index("created_at").resample('ME').agg({"revenue2": 'sum'})

,revenue2
created_at,
2023-01-31 00:00:00+00:00,51636.038174
2023-02-28 00:00:00+00:00,50384.20268
2023-03-31 00:00:00+00:00,52697.171393
2023-04-30 00:00:00+00:00,57201.724886
2023-05-31 00:00:00+00:00,62591.078277
2023-06-30 00:00:00+00:00,57968.263822
2023-07-31 00:00:00+00:00,78143.717445
2023-08-31 00:00:00+00:00,80262.676879
2023-09-30 00:00:00+00:00,79320.929587
